## 特征提取

依赖与环境

In [1]:
import sys, platform, os, time, importlib, subprocess
import mindspore as ms
from mindspore import context

def ensure(pkg, ver=None):
    try:
        m = importlib.import_module(pkg)
        if ver:
            print(f"{pkg}=={getattr(m,'__version__','?')} (要求 {ver})")
        else:
            print(f"{pkg} 已安装")
    except Exception as e:
        print(f"缺失 {pkg}，开始安装……")
        cmd = [sys.executable, "-m", "pip", "install", pkg + (f"=={ver}" if ver else "")]
        print(">>>", " ".join(cmd))
        subprocess.check_call(cmd)

ensure("mindnlp", "0.4.1")
ensure("numpy")

[WARNING] DEVICE(5203,e7ffc65c2020,python):2025-08-22-17:00:49.994.794 [mindspore/ccsrc/utils/dlopen_macro.h:165] DlsymAscend] Dynamically load symbol aclprofGetSupportedFeaturesV2 failed, result = /usr/local/Ascend/ascend-toolkit/latest/lib64/libmsprofiler.so: undefined symbol: aclprofGetSupportedFeaturesV2
[WARNING] DEVICE(5203,e7ffc65c2020,python):2025-08-22-17:00:49.994.921 [mindspore/ccsrc/utils/dlopen_macro.h:165] DlsymAscend] Dynamically load symbol aclrtEventGetTimestamp failed, result = /usr/local/Ascend/ascend-toolkit/latest/lib64/libascendcl.so: undefined symbol: aclrtEventGetTimestamp
/usr/local/miniconda3/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/miniconda3/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  re

mindnlp==? (要求 0.4.1)
numpy 已安装


加载 BGE 模型

In [2]:
from mindnlp.transformers import AutoTokenizer, AutoModel

MODEL_ID = "BAAI/bge-small-zh-v1.5"  
print("Loading:", MODEL_ID)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)   
model = AutoModel.from_pretrained(MODEL_ID)
model.set_train(False)

Loading: BAAI/bge-small-zh-v1.5


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(21128, 512, padding_idx=0)
    (position_embeddings): Embedding(512, 512)
    (token_type_embeddings): Embedding(2, 512)
    (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-3): 4 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear (512 -> 512)
            (key): Linear (512 -> 512)
            (value): Linear (512 -> 512)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear (512 -> 512)
            (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (intermediate): BertIntermediate(
          (dense): Linear (512 -> 2048)
          (intermediate_act_fn):

文本特征提取的编码函数

步骤如下：

（1）输入：文本列表 → tokenizer → token 张量

（2）模型输出：每个 token 的向量 [B, L, H]（B = 句子数，L = 每句子 token 数，H = 向量维度）

（3）平均池化：每句子的向量 = 所有 token 向量的平均值（求和 ÷ token 数 → [B, H]）

（4）归一化：把每个向量缩放到长度为1，使得点积=余弦相似度

（5）输出：每句子一个固定维度向量 [N, H]

In [3]:
import numpy as np
import mindspore.ops as ops

def encode_texts(texts, max_len=256, batch_size=16):
    all_vecs = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        enc = tokenizer(batch, padding=True, truncation=True, max_length=max_len, return_tensors="ms")
        out = model(**enc)
        mask = enc["attention_mask"].astype(ms.float32).expand_dims(-1) 
        summed = ops.sum(out.last_hidden_state * mask, dim=1, keepdim=False)  
        counts = ops.clip_by_value(ops.sum(mask, dim=1, keepdim=False), 1.0, 1e9)
        mean_pooled = summed / counts  
        all_vecs.append(mean_pooled.asnumpy())
    X = np.vstack(all_vecs).astype("float32")
    X /= (np.linalg.norm(X, axis=1, keepdims=True) + 1e-12)
    return X

准备语料

In [4]:
corpus = [
    "我喜欢在假期去海边旅行，看看大海、晒晒太阳。",
    "家里新来了一只可爱的猫咪，特别黏人。",
    "最近在学深度学习，准备做一个图像检索的小项目。",
    "今天读到一句话：读书破万卷，下笔如有神。",
    "科学技术是第一生产力。",
    "周末打算去爬山，呼吸新鲜空气，顺便拍点照片。",
    "宠物狗很听话，已经学会了坐下和握手。",
    "论文要交了，我在写实验部分和结果分析。",
]
corpus_embeds = encode_texts(corpus)
print("语料向量形状:", corpus_embeds.shape)

语料向量形状: (8, 512)


语义检索：给定查询，找到最相似的句子 Top-K

In [5]:
def search_topk(query_texts, index_texts, index_embeds, topk=3):
    q_emb = encode_texts(query_texts)
    sims = q_emb @ index_embeds.T    
    results = []
    for qi, q in enumerate(query_texts):
        k = min(topk, sims.shape[1])
        cand = np.argpartition(-sims[qi], range(k))[:k]
        cand = cand[np.argsort(-sims[qi, cand])]  
        items = [(int(j), float(sims[qi, j]), index_texts[j]) for j in cand]
        results.append({"query": q, "topk": items})
    return results

In [6]:
queries = [
    "找关于旅游出行的句子",
    "和科研学习相关的内容",
    "与宠物有关"
]

res = search_topk(queries, corpus, corpus_embeds, topk=3)

print("\n=== 语义检索结果（Top-3） ===")
for r in res:
    print(f"\nQuery: {r['query']}")
    for rank, (idx, s, txt) in enumerate(r["topk"], 1):
        print(f"  {rank}. score={s:.4f} | #{idx} | {txt}")


=== 语义检索结果（Top-3） ===

Query: 找关于旅游出行的句子
  1. score=0.4897 | #0 | 我喜欢在假期去海边旅行，看看大海、晒晒太阳。
  2. score=0.3406 | #5 | 周末打算去爬山，呼吸新鲜空气，顺便拍点照片。
  3. score=0.2899 | #2 | 最近在学深度学习，准备做一个图像检索的小项目。

Query: 和科研学习相关的内容
  1. score=0.5057 | #2 | 最近在学深度学习，准备做一个图像检索的小项目。
  2. score=0.4604 | #4 | 科学技术是第一生产力。
  3. score=0.4596 | #7 | 论文要交了，我在写实验部分和结果分析。

Query: 与宠物有关
  1. score=0.4877 | #6 | 宠物狗很听话，已经学会了坐下和握手。
  2. score=0.4686 | #1 | 家里新来了一只可爱的猫咪，特别黏人。
  3. score=0.4074 | #2 | 最近在学深度学习，准备做一个图像检索的小项目。
